In [10]:
from time import time
import matplotlib.pyplot as plt
import numpy as np


In [14]:
from particle_markov_chain_monte_carlo import Particle_marginal_Metropolis_Hastings, dati
from scipy.optimize import minimize,  differential_evolution, fmin
from scipy.stats import norm, multivariate_normal

class Monte_Carlo_Expectation_Maximization(dati):
    def __init__(self, turni_da_ottimizzare = 30, campioni_monte_carlo = 2):
        self.M = turni_da_ottimizzare
        self.MC = campioni_monte_carlo
        self.theta_history = np.zeros((6,self.M))
        self.log_lik_history = np.zeros(self.M)
        
        self.PMCMC = Particle_marginal_Metropolis_Hastings(number_of_iterations=300)  #number_of_iterations=1000
        
        dati.__init__(self); 
        
        self.PhiALL = np.zeros((self.MC, 6, 2))
        self.XALL = np.zeros((self.MC, 6, 18))
        
        self.PARMIN = [0.01, 1e-5, 0.002, 1e-4, 0.0001, 0.3e-5] ; self.PARMAX = [1, 1e-1, 0.5, 0.5, 0.1, 5e-5]
        self.bounds = []
        for i in range(6):
            self.bounds.append(tuple([self.PARMIN[i], self.PARMAX[i]]))
        
        
    def run(self, theta_init):
        self.theta_choose = theta_init
        self.phi_subject = np.array(theta_init[:2])
        
        for em in range(self.M):
            print("\n### EM iterazione : ", em)
            for iterazione in range(0, self.MC):
                for subject in range(1,7):
                    if iterazione > 0:
                        self.phi_subject = self.PhiALL[iterazione-1, subject-1, :]
                    phi, X = self.PMCMC.sample_PMCMC(subject, self.theta_choose, self.phi_subject)
                    self.PhiALL[iterazione, subject-1, :] = phi  #self.PMCMC.phi_PMCMC[-1,:] ; 

                    len_subj = len(X)
                    self.XALL[iterazione, subject-1, :len_subj] = X   #self.PMCMC.X_PMCMC[-1,:]



            self.theta_choose = self.ottimizza_MC_approx()
            self.theta_history[:,em] = self.theta_choose
            self.log_lik_history[em] = self.MonteCarlo_Approximation(self.theta_choose)
            
            #if iterazione==0:
             #   self.PMCMC = Particle_marginal_Metropolis_Hastings(number_of_iterations=100)
                
        return self.theta_choose
    
    
    def ottimizza_MC_approx(self):
        
        #                self.iterazione = iterazione
        res = minimize(self.MonteCarlo_Approximation , self.theta_choose, method="Nelder-Mead")
                       
        print(res)
        return res.x
    
    def MonteCarlo_Approximation(self, theta ):
        for i in range(len(theta)):
            if theta[i] > self.PARMAX[i] or theta[i] < self.PARMIN[i]:
                return 1e+300
        MC_log_l = 0
        for mc_ in range(self.MC):
            self.iterazione = mc_
            logPY = self.Compute_LogLikelihood(theta)
            MC_log_l += logPY
        return MC_log_l
        
    def Compute_LogLikelihood(self, theta):
        logpY = 0
        #print(theta)
        for i in range(len(theta)):
            if theta[i] > self.PARMAX[i] or theta[i] < self.PARMIN[i]:
                #print(i, theta[i], self.PARMAX[i], self.PARMIN[i])
                return 1e+300
        try:
            self.phii_distr = multivariate_normal(theta[:2], np.diag(np.array(theta[2:4])**2))
        except:
            print("problema")
            print(theta)
        for subj_ in range(1,7):
            self.select_subj(subj_)

            self.phi_i = self.PhiALL[self.iterazione,subj_ - 1, :]
            self.X_i = self.XALL[self.iterazione, subj_ - 1, :len(self.timei)]
            
            pYigivenXi   = self.ModelCondDensityYgivenXevaluate(self.yobsi[0], self.X_i[0], theta);  
            pXigivenPhii = self.ModelTransitionDensityXevaluate_initial(theta)
            for i_ in range(1, len(self.timei)):
                
                pYijgivenXij = self.ModelCondDensityYgivenXevaluate(self.yobsi[i_], self.X_i[i_], theta) 
                pYigivenXi   *=  pYijgivenXij
                #print(pYigivenXi, pYijgivenXij)
                
                pXijgivenPhii = self.ModelTransitionDensityXevaluate(i_, theta)
                pXigivenPhii = pXigivenPhii * pXijgivenPhii;
                
                
            if pYigivenXi==0:
                pYigivenXi=1e-300
            
            if pXigivenPhii==0:
                pXigivenPhii = 1e-300
                
                
            logpYigivenXi = np.log(pYigivenXi)
            logpXigivenPhii = np.log(pXigivenPhii)     
            logpPhii = np.log(self.ModelAprioriDensityPhievaluate(theta))
            
            logpY += (logpYigivenXi + logpXigivenPhii + logpPhii)
            #print(subj_, logpY, logpYigivenXi, logpXigivenPhii, logpPhii)
            #print("\n\n")
        return -logpY
    
    
    def ModelAprioriDensityPhievaluate(self, theta):
        q = self.phii_distr.pdf(self.phi_i)
        if q==0:
            q = 1e-300
        return q
    
    def ModelCondDensityYgivenXevaluate(self, y_, x_, theta):
        p = norm.pdf(y_, x_, theta[5])
        if p==0: 
            p = 1e-300
        return p
    
    def ModelTransitionDensityXevaluate_initial(self, theta):
        beta = theta[4]
        time1 = self.timei[0]; time0 = 0
        x_t1 = 1e-7
        time = time0
        deltat =  (time1 - time0)/ 100
        while time < time1:
            if (time1-time)>deltat:
                dt = deltat
            else:
                dt = time1-time
            x_t1 += self.ModelDrift(x_t1 )* dt
            time += dt
        uj = x_t1
        sj = beta * uj * np.sqrt(self.timei[0])

        p = norm.pdf(self.X_i[0] ,  uj, sj)
        if p == 0:
            p = 1e-300      
            
        #print(p, uj, sj)
        return p
    
 
        
    
    def ModelTransitionDensityXevaluate(self, time_, theta):
        beta = theta[4]
        
        uj = self.EulerStep(time_)
        sj = beta * uj * np.sqrt(self.timei[time_] - self.timei[time_- 1])

        p = norm.pdf(self.X_i[time_] ,  uj, sj)
        if p == 0:
            p = 1e-300      
            
        #print(p, uj, sj)
        return p
    
    def EulerStep(self, time_):
        time1 = self.timei[time_]; time0 = self.timei[time_ - 1]
        x_t1 = self.X_i[time_ -1]
        time = time0
        deltat =  (time1 - time0)/ 100
        while time < time1:
            if (time1-time)>deltat:
                dt = deltat
            else:
                dt = time1-time
            x_t1 += self.ModelDrift(x_t1 )* dt
            time += dt
        return x_t1
    
    def ModelDrift(self, X):
        return self.phi_i[0]* np.log(self.phi_i[1]/X)* X
    

In [ ]:
start = time()

theta = [0.089, 0.0034, 0.2 , 0.2, 0.1, 5e-5] 
MCEM = Monte_Carlo_Expectation_Maximization()

MCEM.run(theta)
print("tempo esecuzione : %.5f "% (time()-start))


### EM iterazione :  0
	acceptance rate = 0.2700  for subject 1
	phi scelti: (1.017e-01 , 2.093e-03)  marginal y 5.2235e+77
	acceptance rate = 0.3000  for subject 2
	phi scelti: (1.080e-01 , 2.849e-03)  marginal y 1.9122e+67
	acceptance rate = 0.5400  for subject 3
	phi scelti: (1.260e-01 , 4.781e-04)  marginal y 7.4952e+83
	acceptance rate = 0.3633  for subject 4
	phi scelti: (1.176e-01 , 1.472e-03)  marginal y 2.5755e+62
	acceptance rate = 0.2200  for subject 5
	phi scelti: (1.141e-01 , 1.741e-03)  marginal y 3.2946e+91
	acceptance rate = 0.4933  for subject 6
	phi scelti: (1.357e-01 , 6.168e-04)  marginal y 1.2627e+63
	acceptance rate = 0.3600  for subject 1
	phi scelti: (1.168e-01 , 1.442e-03)  marginal y 2.8121e+77
	acceptance rate = 0.3033  for subject 2
	phi scelti: (1.053e-01 , 3.089e-03)  marginal y 1.8960e+67
	acceptance rate = 0.5267  for subject 3
	phi scelti: (1.332e-01 , 4.242e-04)  marginal y 7.1125e+83
	acceptance rate = 0.3933  for subject 4
	phi scelti: (1.184e-01 , 

In [14]:
a = np.array([3.44446436e-01, 1.00032231e-05])
v = np.diag(a**2)
print(v)
multivariate_normal(a,v)

[[1.18643347e-01 0.00000000e+00]
 [0.00000000e+00 1.00064472e-10]]


In [17]:
for i_ in range(0, len(SAEM.timei)):
    print(SAEM.ModelCondDensityYgivenXevaluate(SAEM.yobsi[i_], SAEM.X_i[i_], theta), SAEM.yobsi[i_], SAEM.X_i[i_] )

3988.732773340162 1.96e-06 1e-07
3977.2404065395585 8.1e-06 2.790714452729808e-07
3966.3380607339523 1.16e-05 8.266127858042304e-07
3944.75379119379 2.09e-05 5.89332296414592e-06
3809.4213574947044 3.17e-05 1.3127471270614917e-06
3580.1674469118648 5.57e-05 9.173236177406017e-06
3606.1207448308946 7.03e-05 2.5352485909584112e-05
3299.222991369751 0.000105 4.336241792509879e-05
2384.981915622893 0.00015 4.856479723233464e-05
2586.5151558918506 0.00022 0.00012690487770768922
3488.5931354044405 0.000226 0.00017420269931519967
3912.06944409895 0.000252 0.00023221108582295485


In [18]:
SAEM.yobsi, SAEM.X_i, SAEM.timei

(array([1.96e-06, 8.10e-06, 1.16e-05, 2.09e-05, 3.17e-05, 5.57e-05,
        7.03e-05, 1.05e-04, 1.50e-04, 2.20e-04, 2.26e-04, 2.52e-04]),
 array([1.00000000e-07, 2.79071445e-07, 8.26612786e-07, 5.89332296e-06,
        1.31274713e-06, 9.17323618e-06, 2.53524859e-05, 4.33624179e-05,
        4.85647972e-05, 1.26904878e-04, 1.74202699e-04, 2.32211086e-04]),
 array([ 4.4526,  5.6351,  6.4907,  7.6979,  8.4773,  9.2978, 10.692 ,
        11.3167, 12.8289, 14.5346, 15.5006, 16.4679]))

In [20]:
print(theta, SAEM.Compute_LogLikelihood(theta))
print(SAEM.theta_choose, SAEM.Compute_LogLikelihood(SAEM.theta_choose))

[0.3, 0.009, 0.2, 0.3, 0.3, 0.0001] 754.2900883533351
[2.82509615e-02 5.03784905e-05 2.97474419e-01 4.20632943e-05
 5.47894204e-01 2.29341405e-04] inf


C:\Users\Umbertojunior\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in log


In [5]:
from scipy.optimize import differential_evolution

In [4]:
1/((80 - 60)**0.8)

7.28225681210432

In [ ]:
SAEM.bounds

[(0.01, 1),
 (1e-06, 0.1),
 (0.0001, 0.5),
 (1e-06, 1),
 (0.0001, 1),
 (1e-05, 0.001)]

In [ ]:
res = differential_evolution(SAEM.Compute_LogLikelihood , bounds=SAEM.bounds)
               
print(res)    

C:\Users\Umbertojunior\Anaconda3\lib\site-packages\ipykernel_launcher.py:103: RuntimeWarning: divide by zero encountered in log
C:\Users\Umbertojunior\Anaconda3\lib\site-packages\ipykernel_launcher.py:104: RuntimeWarning: divide by zero encountered in log
C:\Users\Umbertojunior\Anaconda3\lib\site-packages\numpy\core\_methods.py:112: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [ ]:
SAEM.iterazione

In [ ]:
PMCMC = Particle_marginal_Metropolis_Hastings(number_of_iterations=1000)
start = time()
PMCMC.sample_PMCMC(1, theta)
print("tempo esecuzione PMCMC : %.4f sec"% (time()-start))

fig , (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(np.arange(PMCMC.NI),PMCMC.phi_PMCMC[:,0])
ax2.plot(np.arange(PMCMC.NI),PMCMC.phi_PMCMC[:,1])
print("marginal Y pmcmc = ", np.max(PMCMC.marginal_pmcmc))

In [ ]:
PMCMC.phi_PMCMC

In [ ]:
ind_max = np.argmax(PMCMC.marginal_pmcmc)
theta2 = [PMCMC.phi_PMCMC[ind_max,0], PMCMC.phi_PMCMC[ind_max,1] , 0.5, 35e-3, 0.1, 1e-5]
PMCMC.fit_SMC(1, theta2)  #np.mean(PMCMC.phi_PMCMC[0,800:])
plt.plot(PMCMC.timei, PMCMC.sampling_a_path().flatten(), "b*", markersize=5, label = "sampled from SMC with phi max")
plt.plot(PMCMC.timei, PMCMC.X_PMCMC[-1,:], "g*", markersize=5, label = "sampled from PMCMC")
plt.plot(PMCMC.timei, np.mean(PMCMC.X_PMCMC, axis=0), "y*", markersize=5, label = "sampled from PMCMC mean")

plt.plot(PMCMC.timei, PMCMC.yobsi, "r*", markersize=12, label = "observed")
plt.legend()

print("\n \nalpha = %.4e  \t  kappa = %.4e"%(PMCMC.phi_PMCMC[ind_max,0], PMCMC.phi_PMCMC[ind_max,1]))
print("marginal Y SMC sampled = ", PMCMC.marginal_y)

print("\n \nalpha = %.4e  \t  kappa = %.4e"%(PMCMC.phi_PMCMC[-1,0], PMCMC.phi_PMCMC[-1,1]))
print("marginal Y PMPCMC sampled = ", PMCMC.marginal_pmcmc[-1])

print("\n \nalpha = %.4e  \t  kappa = %.4e"%(PMCMC.phi_PMCMC[ind_max,0], PMCMC.phi_PMCMC[ind_max,1]))
print("marginal Y PMCMC max = ", PMCMC.marginal_pmcmc[np.argmax(PMCMC.marginal_pmcmc)])


In [ ]:
np.argmax(PMCMC.marginal_pmcmc), PMCMC.marginal_pmcmc[np.argmax(PMCMC.marginal_pmcmc)]

In [ ]:
PMCMC.marginal_pmcmc

In [ ]:
from scipy.optimize import minimize

In [ ]:
fun = lambda x: (x[0] - 1)**2 + (x[1] - 2.5)**2

def fun_2(x):
    return (x[0] - 1)**2 + (x[1] - 2.5)**2

bnds = ((0,None), (0,None))

In [ ]:
red = minimize(fun_2 , (4,0), method="L-BFGS-B", bounds=bnds, options={'gtol': 1e-6, 'disp': True})

In [ ]:
red.x